<a href="https://colab.research.google.com/github/Iddo77/YOLO_drone_detection/blob/main/YOLO_drones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install scikit-learn opencv-python-headless pillow numpy

                                              0.0/38.1 MB ? eta -:--:--
                                              0.3/38.1 MB 8.9 MB/s eta 0:00:05
                                              0.7/38.1 MB 9.2 MB/s eta 0:00:05
     -                                        1.0/38.1 MB 8.3 MB/s eta 0:00:05
     -                                        1.4/38.1 MB 8.2 MB/s eta 0:00:05
     -                                        1.6/38.1 MB 7.2 MB/s eta 0:00:06
     --                                       1.9/38.1 MB 7.1 MB/s eta 0:00:06
     --                                       2.2/38.1 MB 7.2 MB/s eta 0:00:06
     --                                       2.7/38.1 MB 7.4 MB/s eta 0:00:05
     ---                                      3.0/38.1 MB 7.4 MB/s eta 0:00:05
     ---                                      3.3/38.1 MB 7.3 MB/s eta 0:00:05
     ---                                      3.7/38.1 MB 7.5 MB/s eta 0:00:05
     ----                                     4.1/38.1 MB 7

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'D:\\OU Master AI\\Research methods\\Project\\yolo-drones-env\\Lib\\site-packages\\cv2\\cv2.pyd'
Check the permissions.



In [1]:
import os
import random
import shutil
import glob
import cv2
import gc
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image, ImageEnhance
from typing import List, Tuple

In [2]:
# constants
NUM_TRAIN = 500
RANDOM_STATE = 36
# replace with correct root of project 
PROJECT_ROOT = r"D:\OU Master AI\Research methods\Project"

In [3]:
# get the path to our own project
project_path = os.path.join(PROJECT_ROOT, 'YOLO_drone_detection')
print(project_path)
# get a the path to the dataset folder
dataset_path = os.path.join(project_path, "dataset")
print(dataset_path)
# get the path to the folder with the YOLO v7 code
yolo_path = os.path.join(PROJECT_ROOT, "yolov7")
print(yolo_path)

D:\OU Master AI\Research methods\Project\YOLO_drone_detection
D:\OU Master AI\Research methods\Project\YOLO_drone_detection\dataset
D:\OU Master AI\Research methods\Project\yolov7


In [4]:
def create_train_val_test_txt_files():
    # get all image files and shuffle them
    all_images = [f for f in os.listdir(dataset_path) if f.endswith('.jpg')]
    random.shuffle(all_images)

    # Split the dataset into training, validation, and test sets
    train_images = all_images[:NUM_TRAIN]
    test_val_images = all_images[NUM_TRAIN:]

    # split the test_val set into validation and test sets
    val_images, test_images = train_test_split(test_val_images, 
                                               train_size=0.5, 
                                               test_size=0.5, 
                                               random_state=RANDOM_STATE)

    # Write the paths to the train, val, and test text files
    with open(os.path.join(project_path, 'train.txt'), 'w') as f:
        for img in train_images:
            f.write(os.path.join(dataset_path, img) + '\n')

    with open(os.path.join(project_path, 'val.txt'), 'w') as f:
        for img in val_images:
            f.write(os.path.join(dataset_path, img) + '\n')

    with open(os.path.join(project_path, 'test.txt'), 'w') as f:
        for img in test_images:
            f.write(os.path.join(dataset_path, img) + '\n')

In [5]:
def create_custom_yaml_file():
    """
    Creates a custom YAML file for YOLO that specifies the paths to the training, 
    validation, and test sets.
    """
    yaml_text = f"""train: '{os.path.join(project_path, "train.txt")}'
val: '{os.path.join(project_path, "val.txt")}'
test: '{os.path.join(project_path, "test.txt")}'
nc: 1
names: ['drone']
"""
    with open(os.path.join(project_path, 'custom.yaml'), 'w') as f:
        f.write(yaml_text)


In [6]:
create_train_val_test_txt_files()
create_custom_yaml_file()

In [13]:
# finetune the base model using the custom.yaml file created above
os.chdir(yolo_path)
!python train.py --workers 8 --device 0 --batch-size 8 --data "{os.path.join(project_path, 'custom.yaml')}" --img 640 640 --cfg cfg/training/yolov7.yaml --weights './yolov7_training.pt' --name yolov7-custom --hyp data/hyp.scratch.custom.yaml

wandb: Install Weights & Biases for YOLOR logging with 'pip install wandb' (recommended)

autoanchor: Analyzing anchors... anchors/target = 3.30, Best Possible Recall (BPR) = 1.0000
                 all         529         579     0.00367       0.722      0.0083     0.00105
                 all         529         579     0.00923       0.617     0.00697    0.000859
                 all         529         579     0.00365      0.0759     0.00746    0.000991
                 all         529         579     0.00898      0.0345     0.00937     0.00264
                 all         529         579     0.00684       0.325     0.00455    0.000686
                 all         529         579       0.011      0.0691     0.00541    0.000792
                 all         529         579      0.0265      0.0725     0.00871     0.00129
                 all         529         579     0.00315       0.472     0.00245    0.000368
                 all         529         579     0.00507        0.25     0

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
# finetune with 500 epochs instead of 300
os.chdir(yolo_path)
!python train.py --workers 8 --device 0 --batch-size 8 --data "{os.path.join(project_path, 'custom.yaml')}" --img 640 640 --cfg cfg/training/yolov7.yaml --weights './yolov7_training.pt' --name yolov7-custom --hyp data/hyp.scratch.custom.yaml --epochs 500

wandb: Install Weights & Biases for YOLOR logging with 'pip install wandb' (recommended)

autoanchor: Analyzing anchors... anchors/target = 3.24, Best Possible Recall (BPR) = 1.0000
                 all         429         471     0.00982       0.603     0.00693    0.000857
                 all         429         471     0.00351       0.747     0.00375    0.000499
                 all         429         471      0.0194      0.0276      0.0047    0.000733
                 all         429         471      0.0239      0.0234     0.00554    0.000814
                 all         429         471      0.0635       0.034     0.00888     0.00151
                 all         429         471      0.0118       0.151      0.0101     0.00188
                 all         429         471     0.00505       0.168     0.00341    0.000528
                 all         429         471      0.0841      0.0764      0.0264     0.00564
                 all         429         471      0.0285       0.087     0

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
# get a the path to the augmented dataset folder
augmented_dataset_path = os.path.join(project_path, "augmented-dataset")
os.makedirs(augmented_dataset_path, exist_ok=True)
print(augmented_dataset_path)

D:\OU Master AI\Research methods\Project\YOLO_drone_detection\augmented-dataset


In [8]:
def create_augmented_dataset(n=50):
    
    # Read train.txt to get the list of image paths
    with open(os.path.join(project_path, 'train.txt'), 'r') as f:
        image_files = f.read().splitlines()
    

    # Randomly select n images files
    random_image_files = random.sample(image_files, n)

    for file_path in random_image_files:
        filename = os.path.basename(file_path)
        destination_path = os.path.join(augmented_dataset_path, filename)
        
        # Copy the image file
        shutil.copy(file_path, destination_path)
        
        # Get the corresponding bounding box file path
        base = os.path.splitext(file_path)[0]
        bbox_file_path = f"{base}.txt"
        
        # Copy the bounding box file
        shutil.copy(bbox_file_path, augmented_dataset_path)

In [8]:
create_augmented_dataset()

In [9]:
def jitter_image_and_copy_bbox(image: np.ndarray, bbox_path: str, image_name: str) -> None:
    """
    This function applies color jitter to the image, saves the new image, and copies the bounding box file.
    
    Args:
        image: numpy array of shape (height, width, channels).
        bbox_path: path to the bounding box file for the image.
        image_name: name of the image file.
    """
    # Apply color jitter
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # random_bright = .25 + np.random.uniform()
    random_bright = .5 + 1.5 * np.random.uniform()
    image[:, :, 2] = image[:, :, 2] * random_bright
    image = cv2.cvtColor(image, cv2.COLOR_HSV2BGR)

    # Save the new image
    new_image_path = os.path.join(augmented_dataset_path, f"{image_name}_jittered.jpg")
    cv2.imwrite(new_image_path, image)

    # Copy the bounding box file
    new_bbox_path = os.path.join(augmented_dataset_path, f"{image_name}_jittered.txt")
    shutil.copy(bbox_path, new_bbox_path)


In [10]:
def alter_brightness_and_copy_bbox(image: np.ndarray, bbox_path: str, image_name: str, factor: float) -> None:
    """
    This function alters brightness of the image, saves the new image, and copies the bounding box file.
    
    Args:
        image: numpy array of shape (height, width, channels).
        bbox_path: path to the bounding box file for the image.
        image_name: name of the image file.
        factor: brightness control (1.0 gives original image)
    """
    
    # Convert to PIL image and  alter the brightness
    pil_img = Image.fromarray(image)
    enhancer = ImageEnhance.Brightness(pil_img)
    enhanced_im = enhancer.enhance(factor)
    
     # Convert back to numpy and save the new image
    image = np.asarray(enhanced_im)
    new_image_path = os.path.join(augmented_dataset_path, f"{image_name}_brightness_altered.jpg")
    cv2.imwrite(new_image_path, image)
    
    # Copy the bounding box file
    new_bbox_path = os.path.join(augmented_dataset_path, f"{image_name}_brightness_altered.txt")
    shutil.copy(bbox_path, new_bbox_path)


In [11]:
def alter_contrast_and_copy_bbox(image: np.ndarray, bbox_path: str, image_name: str, factor: float) -> None:
    """
    This function alters contrast of the image, saves the new image, and copies the bounding box file.
    
    Args:
        image: numpy array of shape (height, width, channels).
        bbox_path: path to the bounding box file for the image.
        image_name: name of the image file.
        factor: contrast control (1.0 gives original image)
    """
    # Convert to PIL image and alter the contrast
    pil_img = Image.fromarray(image)
    enhancer = ImageEnhance.Contrast(pil_img)
    enhanced_im = enhancer.enhance(factor)

    # Convert back to numpy and save the new image
    image = np.asarray(enhanced_im)
    new_image_path = os.path.join(augmented_dataset_path, f"{image_name}_contrast_altered.jpg")
    cv2.imwrite(new_image_path, image)

    # Copy the bounding box file
    new_bbox_path = os.path.join(augmented_dataset_path, f"{image_name}_contrast_altered.txt")
    shutil.copy(bbox_path, new_bbox_path)

In [12]:
def flip_image_horizontally_and_update_bbox(image: np.ndarray, bbox_path: str, image_name: str) -> None:
    """
    This function flips an image horizontally, saves the new image, and updates the bounding box file.
    
    Args:
        image: numpy array of shape (height, width, channels).
        bbox_path: path to the bounding box file for the image.
        image_name: name of the image file.
    """
    # Flip the image horizontally
    image = cv2.flip(image, 1)

    # Save the new image
    new_image_path = os.path.join(augmented_dataset_path, f"{image_name}_flipped_hori.jpg")
    cv2.imwrite(new_image_path, image)

    # Update the bounding box
    bbox_data = np.loadtxt(bbox_path).reshape(-1, 5)
    bbox_data[:, 1] = 1 - bbox_data[:, 1]  # flip x-center

    # Save the new bounding box file
    new_bbox_path = os.path.join(augmented_dataset_path, f"{image_name}_flipped_hori.txt")
    np.savetxt(new_bbox_path, bbox_data.reshape(-1, 5), fmt=["%d", "%f", "%f", "%f", "%f"])


In [13]:
def flip_image_vertically_and_update_bbox(image: np.ndarray, bbox_path: str, image_name: str) -> None:
    """
    This function flips an image vertically, saves the new image, and updates the bounding box file.
    
    Args:
        image: numpy array of shape (height, width, channels).
        bbox_path: path to the bounding box file for the image.
        image_name: name of the image file.
    """
    # Flip the image vertically
    image = cv2.flip(image, 0)

    # Save the new image
    new_image_path = os.path.join(augmented_dataset_path, f"{image_name}_flipped_vert.jpg")
    cv2.imwrite(new_image_path, image)

    # Update the bounding box
    bbox_data = np.loadtxt(bbox_path).reshape(-1, 5)
    bbox_data[:, 2] = 1 - bbox_data[:, 2]  # flip y-center

    # Save the new bounding box file
    new_bbox_path = os.path.join(augmented_dataset_path, f"{image_name}_flipped_vert.txt")
    np.savetxt(new_bbox_path, bbox_data.reshape(-1, 5), fmt=["%d", "%f", "%f", "%f", "%f"])

In [14]:
def rotate_image_90_and_update_bbox(image: np.ndarray, bbox_path: str, image_name: str, clockwise: bool) -> None:
    """
    This function rotates an image by 90 degrees, saves the new image, and updates the bounding box file.
    
    Args:
        image: numpy array of shape (height, width, channels).
        bbox_path: path to the bounding box file for the image.
        image_name: name of the image file.
        clockwise: If True, rotates the image clockwise. If False, rotates counterclockwise.
    """
    # Get suffix for filenames and rotation factor for OpenCV
    if clockwise:
        suffix = 'clockwise'
        rotation_factor = cv2.ROTATE_90_CLOCKWISE
    else:
        suffix = 'counterclockwise'
        rotation_factor = cv2.ROTATE_90_COUNTERCLOCKWISE

    # Rotate the image
    image = cv2.rotate(image, rotation_factor)
    
    # Save the new image
    new_image_path = os.path.join(augmented_dataset_path, f"{image_name}_rotated_{suffix}.jpg")
    cv2.imwrite(new_image_path, image)

    # Update the bounding box
    bbox_data = np.loadtxt(bbox_path).reshape(-1, 5)
    if clockwise:
        bbox_data[:, [1, 2]] = bbox_data[:, [2, 1]]
        bbox_data[:, [3, 4]] = bbox_data[:, [4, 3]]
        bbox_data[:, 1] = 1 - bbox_data[:, 1]
    else:
        bbox_data[:, [1, 2]] = 1 - bbox_data[:, [2, 1]]
        bbox_data[:, [3, 4]] = bbox_data[:, [4, 3]]
        bbox_data[:, 1] = 1 - bbox_data[:, 1]

    # Save the new bounding box file
    new_bbox_path = os.path.join(augmented_dataset_path, f"{image_name}_rotated_{suffix}.txt")
    np.savetxt(new_bbox_path, bbox_data.reshape(-1, 5), fmt=["%d", "%f", "%f", "%f", "%f"])

In [15]:
def conditional_zoom_and_update_bbox(image: np.ndarray, bbox_path: str, image_name: str) -> None:
    """
    This function conditionally zooms an image, saves the new image, and updates the bounding box file.
    
    Args:
        image: numpy array of shape (height, width, channels).
        bbox_path: path to the bounding box file for the image.
        image_name: name of the image file.
    """
    # Load the bounding box data
    bbox_data = np.loadtxt(bbox_path).reshape(-1, 5)
    
    # Calculate the area of the bounding box and the image
    bbox_area = ((bbox_data[:, 3] - bbox_data[:, 1]) * (bbox_data[:, 4] - bbox_data[:, 2])).sum()

    # Choose zoom direction based on the bounding box area
    zoom_factor = 1.3 if bbox_area > 0.5 else 0.7

    # Resize the image
    image = cv2.resize(image, None, fx=zoom_factor, fy=zoom_factor, interpolation=cv2.INTER_LINEAR)

    # Save the new image
    new_image_path = os.path.join(augmented_dataset_path, f"{image_name}_zoomed.jpg")
    cv2.imwrite(new_image_path, image)

    # Save the new bounding box file
    new_bbox_path = os.path.join(augmented_dataset_path, f"{image_name}_zoomed.txt")
    np.savetxt(new_bbox_path, bbox_data.reshape(-1, 5), fmt=["%d", "%f", "%f", "%f", "%f"])

In [16]:
def conditional_translate_and_update_bbox(image: np.ndarray, bbox_path: str, image_name: str) -> None:
    """
    This function conditionally translates an image, saves the new image, and updates the bounding box file.
    
    Args:
        image: numpy array of shape (height, width, channels).
        bbox_path: path to the bounding box file for the image.
        image_name: name of the image file.
    """
    # Load the bounding box data
    bbox_data = np.loadtxt(bbox_path).reshape(-1, 5)
    
    # Calculate the center of the bounding box
    bbox_center = bbox_data[:, 1:3].mean(axis=0)

    # Define translation distances (10% of image width/height)
    translate_dist = np.array([0.1, 0])  # translate horizontally

    # Choose translation direction based on the bounding box center
    translate_dist *= -1 if bbox_center[0] < 0.5 else 1  # left if center is on the left half, else right

    # Create the translation matrix
    M = np.float32([[1, 0, translate_dist[0]*image.shape[1]], [0, 1, translate_dist[1]*image.shape[0]]])

    # Translate the image
    image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))

    # Update the bounding box center
    bbox_data[:, 1:3] += translate_dist

    # Save the new image
    new_image_path = os.path.join(augmented_dataset_path, f"{image_name}_translated.jpg")
    cv2.imwrite(new_image_path, image)

    # Save the new bounding box file
    new_bbox_path = os.path.join(augmented_dataset_path, f"{image_name}_translated.txt")
    np.savetxt(new_bbox_path, bbox_data.reshape(-1, 5), fmt=["%d", "%f", "%f", "%f", "%f"])

In [17]:
def augment_image(image_path: str) -> None:
    """
    This function applies various augmentations to an image.
    
    Args:
        image_path: Full path to the image file.
    """
    # Load the image
    image = cv2.imread(image_path)

    # Extract the image name (without extension) and bounding box path
    image_name = os.path.splitext(os.path.basename(image_path))[0]
    bbox_path = os.path.join(os.path.dirname(image_path), f"{image_name}.txt")

    # Apply various transformations and save the results
    jitter_image_and_copy_bbox(image, bbox_path, image_name)
    alter_brightness_and_copy_bbox(image, bbox_path, image_name, 1.5)
    alter_contrast_and_copy_bbox(image, bbox_path, image_name, 1.5)
    flip_image_horizontally_and_update_bbox(image, bbox_path, image_name)
    flip_image_vertically_and_update_bbox(image, bbox_path, image_name)
    rotate_image_90_and_update_bbox(image, bbox_path, image_name, clockwise=True)
    rotate_image_90_and_update_bbox(image, bbox_path, image_name, clockwise=False)
    conditional_zoom_and_update_bbox(image, bbox_path, image_name)
    conditional_translate_and_update_bbox(image, bbox_path, image_name)

In [18]:
def augment_all_images() -> None:
    """
    This function finds all images in the augmented_dataset_path directory and applies various augmentations to each image.
    """
    # Find all .jpg image files in the directory
    image_paths = glob.glob(os.path.join(augmented_dataset_path, '*.jpg'))

    # Apply augmentations to each image
    for image_path in image_paths:
        augment_image(image_path)

In [19]:
augment_all_images()

In [20]:
def draw_bounding_boxes(image_path: str, bbox_path: str, output_dir: str) -> None:
    """
    This function draws bounding boxes on an image and saves the image.

    Args:
        image_path: Full path to the image file.
        bbox_path: Full path to the corresponding bounding box file.
        output_dir: Directory to save the images with bounding boxes.
    """
    # Load the image
    image = cv2.imread(image_path)

    # Load the bounding boxes
    bbox_data = np.loadtxt(bbox_path).reshape(-1, 5)

    # Draw each bounding box on the image
    for bbox in bbox_data:
        # Convert bbox from relative to absolute coordinates
        abs_bbox = np.zeros_like(bbox)
        abs_bbox[0] = bbox[0]
        abs_bbox[1] = (bbox[1] - bbox[3] / 2) * image.shape[1]  # x_min
        abs_bbox[2] = (bbox[2] - bbox[4] / 2) * image.shape[0]  # y_min
        abs_bbox[3] = (bbox[1] + bbox[3] / 2) * image.shape[1]  # x_max
        abs_bbox[4] = (bbox[2] + bbox[4] / 2) * image.shape[0]  # y_max

        abs_bbox = abs_bbox.astype(int)

        # Draw the bounding box
        cv2.rectangle(image, (abs_bbox[1], abs_bbox[2]), (abs_bbox[3], abs_bbox[4]), (0, 255, 0), 2)

    # Save the image
    image_name = os.path.basename(image_path)
    output_path = os.path.join(output_dir, image_name)
    cv2.imwrite(output_path, image)

In [21]:
def draw_bboxes_for_all_images_in_folder(input_dir: str, output_dir: str) -> None:
    """
    This function finds all images in the input directory and draws bounding boxes on each image.

    Args:
        input_dir: Directory containing the images and bounding box files.
        output_dir: Directory to save the images with bounding boxes.
    """
    # Find all .jpg image files in the directory
    image_paths = glob.glob(os.path.join(input_dir, '*.jpg'))

    # For each image, draw bounding boxes and save the image
    for image_path in image_paths:
        # Derive the bounding box file path from the image file path
        bbox_path = image_path.rsplit('.', 1)[0] + '.txt'
        draw_bounding_boxes(image_path, bbox_path, output_dir)

In [9]:
dataset_validation_path = os.path.join(os.path.dirname(augmented_dataset_path), "augmented-dataset-validation")

if os.path.exists(dataset_validation_path):
    shutil.rmtree(dataset_validation_path)

os.mkdir(dataset_validation_path)

draw_bboxes_for_all_images_in_folder(augmented_dataset_path, dataset_validation_path)

NameError: name 'draw_bboxes_for_all_images_in_folder' is not defined

In [34]:
# load all augmented image paths
all_images = [f for f in os.listdir(augmented_dataset_path) if f.endswith('.jpg')]
random.shuffle(all_images)

# create a train-augmented.txt file with all augmented image paths
with open(os.path.join(project_path, 'train-augmented.txt'), 'w') as f:
    for img in all_images:
        f.write(os.path.join(augmented_dataset_path, img) + '\n')

# copy custom.yaml as custom-augmented.yaml
custom_yaml_path = os.path.join(project_path, 'custom.yaml')
new_custom_yaml_path = os.path.join(project_path, 'custom-augmented.yaml')
shutil.copy(custom_yaml_path, new_custom_yaml_path)

# replace 'train.txt' with 'train-augmented.txt' in custom-augmented.yaml
with open(new_custom_yaml_path, 'r') as file:
    content = file.read()

new_content = content.replace('train.txt', 'train-augmented.txt')

with open(new_custom_yaml_path, 'w') as file:
    file.write(new_content)


In [142]:
# finetune augmented dataset with 500 epochs
os.chdir(yolo_path)
!python train.py --workers 8 --device 0 --batch-size 6 --data "{new_custom_yaml_path}" --img 640 640 --cfg cfg/training/yolov7.yaml --weights './yolov7_training.pt' --name yolov7-custom --hyp data/hyp.scratch.custom.yaml --epochs 500

wandb: Install Weights & Biases for YOLOR logging with 'pip install wandb' (recommended)

autoanchor: Analyzing anchors... anchors/target = 3.29, Best Possible Recall (BPR) = 1.0000
                 all         429         471      0.0142       0.618      0.0121     0.00187
                 all         429         471      0.0105       0.191     0.00885     0.00153
                 all         429         471      0.0172       0.197     0.00719     0.00115
                 all         429         471      0.0285      0.0701     0.00953     0.00155
                 all         429         471      0.0961      0.0998      0.0275     0.00483
                 all         429         471      0.0177      0.0255     0.00458    0.000703
                 all         429         471      0.0238      0.0403     0.00583    0.000983
                 all         429         471      0.0337       0.125      0.0115     0.00215
                 all         429         471      0.0154       0.268      

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
def delete_cache_files():
    cache_files = glob.glob(os.path.join(project_path, '**', '*.cache'), recursive=True)
    for file in cache_files:
        os.remove(file)

In [8]:
def cleanup():
    # delete created folders and files
    if os.path.exists(augmented_dataset_path):
        shutil.rmtree(augmented_dataset_path)
    if os.path.exists(dataset_validation_path):
        shutil.rmtree(dataset_validation_path)
    delete_cache_files()
    yaml_file = os.path.join(project_path, 'custom-augmented.yaml')
    if os.path.exists(yaml_file):
        os.remove(yaml_file)
    train_file = os.path.join(project_path, 'train-augmented.txt')
    if os.path.exists(train_file):
        os.remove(train_file)
    # garbage collection
    gc.collect()
    # clear CUDA memory
    torch.cuda.empty_cache()

In [9]:
cleanup()

NameError: name 'augmented_dataset_path' is not defined

In [33]:
# create an dataset of 100 images augmented to 1000
os.makedirs(augmented_dataset_path, exist_ok=True)
create_augmented_dataset(n=100)
augment_all_images()
# create files fort visual verification
os.mkdir(dataset_validation_path)
draw_bboxes_for_all_images_in_folder(augmented_dataset_path, dataset_validation_path)

In [36]:
# finetune augmented dataset with 500 epochs
os.chdir(yolo_path)
!python train.py --workers 8 --device 0 --batch-size 6 --data "{new_custom_yaml_path}" --img 640 640 --cfg cfg/training/yolov7.yaml --weights './yolov7_training.pt' --name yolov7-custom --hyp data/hyp.scratch.custom.yaml --epochs 500

wandb: Install Weights & Biases for YOLOR logging with 'pip install wandb' (recommended)

autoanchor: Analyzing anchors... anchors/target = 3.43, Best Possible Recall (BPR) = 1.0000
                 all         429         471     0.00788      0.0127     0.00382    0.000966
                 all         429         471      0.0231       0.161     0.00724     0.00117
                 all         429         471      0.0545      0.0849      0.0115     0.00198
                 all         429         471      0.0515       0.106      0.0168     0.00304
                 all         429         471      0.0198      0.0382     0.00377    0.000711
                 all         429         471      0.0801       0.113      0.0352      0.0061
                 all         429         471       0.132      0.0531      0.0236     0.00333
                 all         429         471      0.0611        0.24       0.036     0.00543
                 all         429         471      0.0643       0.214      

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



                 all         429         471       0.813       0.703       0.775       0.407
                 all         429         471       0.784       0.694        0.75       0.401
                 all         429         471       0.834       0.694       0.771       0.419
                 all         429         471       0.868       0.686       0.778       0.421
                 all         429         471       0.806       0.724       0.773       0.421
                 all         429         471       0.861       0.698        0.79       0.431
                 all         429         471       0.831       0.713       0.786       0.429
                 all         429         471       0.822        0.69       0.778       0.416
                 all         429         471       0.847       0.683       0.781       0.412
                 all         429         471       0.809       0.694       0.751       0.412
                 all         429         471       0.783        0.73  

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
def copy_train_val_test_files(train_val_test_path: str):
    """
    Copy files from the train/val/test folder to the project folder.

    Args:
        train_val_test_path (str): The path to the folder containining custom.yaml, train.txt, val.txt and test.txt
    """
    for filename in os.listdir(train_val_test_path):
        source_file = os.path.join(train_val_test_path, filename)
        destination_file = os.path.join(project_path, filename)
        # Copy the file to the folder and overwrite if it already exists
        shutil.copy2(source_file, destination_file)

In [14]:
# prepare testing the trained model of 300 images and 300 epochs
os.chdir(yolo_path)
train_val_test_folder = r'D:\OU Master AI\Research methods\Project\YOLO_drone_detection\train_val_test_300' 
copy_train_val_test_files(train_val_test_folder)
weights_folder = r'D:\OU Master AI\Research methods\Project\YOLO_drone_detection\results_300_img_300_epochs_training\weights'
best_weights = os.path.join(weights_folder, 'best.pt')
last_weights = os.path.join(weights_folder, 'last.pt')

In [15]:
# test the best weights of  the trained model of 300 images and 300 epochs
delete_cache_files()
!python test.py --data "{os.path.join(project_path, 'custom.yaml')}" --img 640 --save-json --single-cls --batch 6 --conf 0.001 --iou 0.65 --device 0 --weights "{best_weights}" --task test --name test_yolov7_300_300_best

Namespace(weights=['D:\\OU Master AI\\Research methods\\Project\\YOLO_drone_detection\\results_300_img_300_epochs_training\\weights\\best.pt'], data='D:\\OU Master AI\\Research methods\\Project\\YOLO_drone_detection\\custom.yaml', batch_size=6, img_size=640, conf_thres=0.001, iou_thres=0.65, task='test', device='0', single_cls=True, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project='runs/test', name='test_yolov7_300_300_best', exist_ok=False, no_trace=False, v5_metric=False)
Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

                 all         530         569       0.752       0.704       0.754       0.373
Speed: 12.3/1.5/13.8 ms inference/NMS/total per 640x640 image at batch-size 6

Evaluating pycocotools mAP... saving runs\test\test_yolov7_300_300_best2\best_predictions.json...


YOLOR  2023-5-7 torch 2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6143.5MB)

D:\OU Master AI\Research methods\Project\yolo-drones-env\Lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36481772 parameters, 6194944 gradients, 103.2 GFLOPS

Scanning images:   0%|          | 0/530 [00:00<?, ?it/s]
test: Scanning 'D:\OU Master AI\Research methods\Project\YOLO_drone_detection\test' images and labels... 159 found, 0 missing, 0 empty, 0 corrupted:  30%|###       | 159/530 [00:00<00:00, 1582.46it/s]
test: Scanning 'D:\OU Master AI\Research methods\Project\YOLO_drone_detection\test' images and labels... 381 found, 0 missing, 0 empty, 0 corrupted:  72%|#######1  | 381/530 [00:00<00:00, 1948.25it/s]
test: Scanning 'D:\OU Ma

In [16]:
# test the last weights of  the trained model of 300 images and 300 epochs
delete_cache_files()
!python test.py --data "{os.path.join(project_path, 'custom.yaml')}" --img 640 --save-json --single-cls --batch 6 --conf 0.001 --iou 0.65 --device 0 --weights "{last_weights}" --task test --name test_yolov7_300_300_last

Namespace(weights=['D:\\OU Master AI\\Research methods\\Project\\YOLO_drone_detection\\results_300_img_300_epochs_training\\weights\\last.pt'], data='D:\\OU Master AI\\Research methods\\Project\\YOLO_drone_detection\\custom.yaml', batch_size=6, img_size=640, conf_thres=0.001, iou_thres=0.65, task='test', device='0', single_cls=True, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project='runs/test', name='test_yolov7_300_300_last', exist_ok=False, no_trace=False, v5_metric=False)
Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

                 all         530         569       0.745       0.677       0.731       0.364
Speed: 12.5/1.5/14.1 ms inference/NMS/total per 640x640 image at batch-size 6

Evaluating pycocotools mAP... saving runs\test\test_yolov7_300_300_last2\last_predictions.json...


YOLOR  2023-5-7 torch 2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6143.5MB)

D:\OU Master AI\Research methods\Project\yolo-drones-env\Lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36481772 parameters, 6194944 gradients, 103.2 GFLOPS

Scanning images:   0%|          | 0/530 [00:00<?, ?it/s]
test: Scanning 'D:\OU Master AI\Research methods\Project\YOLO_drone_detection\test' images and labels... 160 found, 0 missing, 0 empty, 0 corrupted:  30%|###       | 160/530 [00:00<00:00, 1591.34it/s]
test: Scanning 'D:\OU Master AI\Research methods\Project\YOLO_drone_detection\test' images and labels... 386 found, 0 missing, 0 empty, 0 corrupted:  73%|#######2  | 386/530 [00:00<00:00, 1972.26it/s]
test: Scanning 'D:\OU Ma

In [17]:
# prepare testing the trained model of 300 images and 500 epochs
weights_folder = r'D:\OU Master AI\Research methods\Project\YOLO_drone_detection\results_300_img_500_epochs_training\weights'
best_weights = os.path.join(weights_folder, 'best.pt')
last_weights = os.path.join(weights_folder, 'last.pt')

In [19]:
# test the best weights of  the trained model of 300 images and 500 epochs
delete_cache_files()
!python test.py --data "{os.path.join(project_path, 'custom.yaml')}" --img 640 --save-json --single-cls --batch 6 --conf 0.001 --iou 0.65 --device 0 --weights "{best_weights}" --task test --name test_yolov7_300_500_best

Namespace(weights=['D:\\OU Master AI\\Research methods\\Project\\YOLO_drone_detection\\results_300_img_500_epochs_training\\weights\\best.pt'], data='D:\\OU Master AI\\Research methods\\Project\\YOLO_drone_detection\\custom.yaml', batch_size=6, img_size=640, conf_thres=0.001, iou_thres=0.65, task='test', device='0', single_cls=True, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project='runs/test', name='test_yolov7_300_500_best', exist_ok=False, no_trace=False, v5_metric=False)
Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

                 all         530         569        0.87       0.824       0.877       0.505
Speed: 12.1/1.5/13.6 ms inference/NMS/total per 640x640 image at batch-size 6

Evaluating pycocotools mAP... saving runs\test\test_yolov7_300_500_best2\best_predictions.json...


YOLOR  2023-5-7 torch 2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6143.5MB)

D:\OU Master AI\Research methods\Project\yolo-drones-env\Lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36481772 parameters, 6194944 gradients, 103.2 GFLOPS

Scanning images:   0%|          | 0/530 [00:00<?, ?it/s]
test: Scanning 'D:\OU Master AI\Research methods\Project\YOLO_drone_detection\test' images and labels... 172 found, 0 missing, 0 empty, 0 corrupted:  32%|###2      | 172/530 [00:00<00:00, 1716.15it/s]
test: Scanning 'D:\OU Master AI\Research methods\Project\YOLO_drone_detection\test' images and labels... 411 found, 0 missing, 0 empty, 0 corrupted:  78%|#######7  | 411/530 [00:00<00:00, 2095.90it/s]
test: Scanning 'D:\OU Ma

In [20]:
# test the last weights of  the trained model of 300 images and 500 epochs
delete_cache_files()
!python test.py --data "{os.path.join(project_path, 'custom.yaml')}" --img 640 --save-json --single-cls --batch 6 --conf 0.001 --iou 0.65 --device 0 --weights "{last_weights}" --task test --name test_yolov7_300_500_last

Namespace(weights=['D:\\OU Master AI\\Research methods\\Project\\YOLO_drone_detection\\results_300_img_500_epochs_training\\weights\\last.pt'], data='D:\\OU Master AI\\Research methods\\Project\\YOLO_drone_detection\\custom.yaml', batch_size=6, img_size=640, conf_thres=0.001, iou_thres=0.65, task='test', device='0', single_cls=True, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project='runs/test', name='test_yolov7_300_500_last', exist_ok=False, no_trace=False, v5_metric=False)
Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

                 all         530         569       0.859       0.821       0.867       0.491
Speed: 12.4/1.5/13.9 ms inference/NMS/total per 640x640 image at batch-size 6

Evaluating pycocotools mAP... saving runs\test\test_yolov7_300_500_last2\last_predictions.json...


YOLOR  2023-5-7 torch 2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6143.5MB)

D:\OU Master AI\Research methods\Project\yolo-drones-env\Lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36481772 parameters, 6194944 gradients, 103.2 GFLOPS

Scanning images:   0%|          | 0/530 [00:00<?, ?it/s]
test: Scanning 'D:\OU Master AI\Research methods\Project\YOLO_drone_detection\test' images and labels... 162 found, 0 missing, 0 empty, 0 corrupted:  31%|###       | 162/530 [00:00<00:00, 1616.01it/s]
test: Scanning 'D:\OU Master AI\Research methods\Project\YOLO_drone_detection\test' images and labels... 390 found, 0 missing, 0 empty, 0 corrupted:  74%|#######3  | 390/530 [00:00<00:00, 1998.70it/s]
test: Scanning 'D:\OU Ma

In [21]:
# prepare testing the trained model of 500 images and 500 epochs
train_val_test_folder = r'D:\OU Master AI\Research methods\Project\YOLO_drone_detection\train_val_test_500' 
copy_train_val_test_files(train_val_test_folder)
weights_folder = r'D:\OU Master AI\Research methods\Project\YOLO_drone_detection\results_500_img_500_epochs_training\weights'
best_weights = os.path.join(weights_folder, 'best.pt')
last_weights = os.path.join(weights_folder, 'last.pt')

In [22]:
# test the best weights of  the trained model of 500 images and 500 epochs
delete_cache_files()
!python test.py --data "{os.path.join(project_path, 'custom.yaml')}" --img 640 --save-json --single-cls --batch 6 --conf 0.001 --iou 0.65 --device 0 --weights "{best_weights}" --task test --name test_yolov7_500_500_best

Namespace(weights=['D:\\OU Master AI\\Research methods\\Project\\YOLO_drone_detection\\results_500_img_500_epochs_training\\weights\\best.pt'], data='D:\\OU Master AI\\Research methods\\Project\\YOLO_drone_detection\\custom.yaml', batch_size=6, img_size=640, conf_thres=0.001, iou_thres=0.65, task='test', device='0', single_cls=True, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project='runs/test', name='test_yolov7_500_500_best', exist_ok=False, no_trace=False, v5_metric=False)
Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

                 all         430         465       0.925       0.877       0.921       0.562
Speed: 13.3/1.5/14.8 ms inference/NMS/total per 640x640 image at batch-size 6

Evaluating pycocotools mAP... saving runs\test\test_yolov7_500_500_best\best_predictions.json...
p

YOLOR  2023-5-7 torch 2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6143.5MB)

D:\OU Master AI\Research methods\Project\yolo-drones-env\Lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36481772 parameters, 6194944 gradients, 103.2 GFLOPS

Scanning images:   0%|          | 0/430 [00:00<?, ?it/s]
test: Scanning 'D:\OU Master AI\Research methods\Project\YOLO_drone_detection\test' images and labels... 44 found, 0 missing, 0 empty, 0 corrupted:  10%|#         | 44/430 [00:00<00:00, 431.89it/s]
test: Scanning 'D:\OU Master AI\Research methods\Project\YOLO_drone_detection\test' images and labels... 88 found, 0 missing, 0 empty, 0 corrupted:  20%|##        | 88/430 [00:00<00:00, 390.47it/s]
test: Scanning 'D:\OU Master A

In [23]:
# test the best weights of  the trained model of 500 images and 500 epochs
delete_cache_files()
!python test.py --data "{os.path.join(project_path, 'custom.yaml')}" --img 640 --save-json --single-cls --batch 6 --conf 0.001 --iou 0.65 --device 0 --weights "{last_weights}" --task test --name test_yolov7_500_500_last

Namespace(weights=['D:\\OU Master AI\\Research methods\\Project\\YOLO_drone_detection\\results_500_img_500_epochs_training\\weights\\last.pt'], data='D:\\OU Master AI\\Research methods\\Project\\YOLO_drone_detection\\custom.yaml', batch_size=6, img_size=640, conf_thres=0.001, iou_thres=0.65, task='test', device='0', single_cls=True, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project='runs/test', name='test_yolov7_500_500_last', exist_ok=False, no_trace=False, v5_metric=False)
Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

                 all         430         465       0.926       0.867       0.924       0.568
Speed: 12.7/1.4/14.2 ms inference/NMS/total per 640x640 image at batch-size 6

Evaluating pycocotools mAP... saving runs\test\test_yolov7_500_500_last\last_predictions.json...
p

YOLOR  2023-5-7 torch 2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6143.5MB)

D:\OU Master AI\Research methods\Project\yolo-drones-env\Lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36481772 parameters, 6194944 gradients, 103.2 GFLOPS

Scanning images:   0%|          | 0/430 [00:00<?, ?it/s]
test: Scanning 'D:\OU Master AI\Research methods\Project\YOLO_drone_detection\test' images and labels... 168 found, 0 missing, 0 empty, 0 corrupted:  39%|###9      | 168/430 [00:00<00:00, 1670.44it/s]
test: Scanning 'D:\OU Master AI\Research methods\Project\YOLO_drone_detection\test' images and labels... 386 found, 0 missing, 0 empty, 0 corrupted:  90%|########9 | 386/430 [00:00<00:00, 1947.11it/s]
test: Scanning 'D:\OU Ma

## Testing augmented datasets
The difference between last and best weights are quite small. However, it is difficult to determine what epoch is considered the best. To be consistent, we will only compare the weights of the last epoch, so that all metrics are known exacltly. Therefore, only the last weights of the the augmented datasets will be tested.

In [14]:
# prepare testing the trained model of 50 images augmented to 500
train_val_test_folder = r'D:\OU Master AI\Research methods\Project\YOLO_drone_detection\train_val_test_50_to_500' 
copy_train_val_test_files(train_val_test_folder)
weights_folder = r'D:\OU Master AI\Research methods\Project\YOLO_drone_detection\results_50_augmented_to_500\weights'
last_weights = os.path.join(weights_folder, 'last.pt')

In [19]:
# test the best weights of  the trained model of 50 images augmented to 500
os.chdir(yolo_path)
delete_cache_files()
!python test.py --data "{os.path.join(project_path, 'custom.yaml')}" --img 640 --save-json --single-cls --batch 6 --conf 0.001 --iou 0.65 --device 0 --weights "{last_weights}" --task test --name test_yolov7_50_to_500

Namespace(weights=['D:\\OU Master AI\\Research methods\\Project\\YOLO_drone_detection\\results_500_img_500_epochs_training\\weights\\last.pt'], data='D:\\OU Master AI\\Research methods\\Project\\YOLO_drone_detection\\custom.yaml', batch_size=6, img_size=640, conf_thres=0.001, iou_thres=0.65, task='test', device='0', single_cls=True, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project='runs/test', name='test_yolov7_50_to_500', exist_ok=False, no_trace=False, v5_metric=False)
Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

                 all         430         465       0.926       0.867       0.924       0.568
Speed: 13.6/1.7/15.3 ms inference/NMS/total per 640x640 image at batch-size 6

Evaluating pycocotools mAP... saving runs\test\test_yolov7_50_to_500\last_predictions.json...
pycocot

YOLOR  2023-5-7 torch 2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6143.5MB)

D:\OU Master AI\Research methods\Project\yolo-drones-env\Lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36481772 parameters, 6194944 gradients, 103.2 GFLOPS

Scanning images:   0%|          | 0/430 [00:00<?, ?it/s]
test: Scanning 'D:\OU Master AI\Research methods\Project\YOLO_drone_detection\test' images and labels... 14 found, 0 missing, 0 empty, 0 corrupted:   3%|3         | 14/430 [00:00<00:03, 133.67it/s]
test: Scanning 'D:\OU Master AI\Research methods\Project\YOLO_drone_detection\test' images and labels... 30 found, 0 missing, 0 empty, 0 corrupted:   7%|6         | 30/430 [00:00<00:02, 146.45it/s]
test: Scanning 'D:\OU Master A

In [23]:
# prepare testing the trained model of 100 images augmented to 1000
train_val_test_folder = r'D:\OU Master AI\Research methods\Project\YOLO_drone_detection\train_val_test_100_to_1000' 
copy_train_val_test_files(train_val_test_folder)
weights_folder = r'D:\OU Master AI\Research methods\Project\YOLO_drone_detection\results_100_augmented_to_1000\weights'
last_weights = os.path.join(weights_folder, 'last.pt')

In [24]:
# test the best weights of  the trained model of 100 images augmented to 1000
os.chdir(yolo_path)
delete_cache_files()
!python test.py --data "{os.path.join(project_path, 'custom.yaml')}" --img 640 --save-json --single-cls --batch 6 --conf 0.001 --iou 0.65 --device 0 --weights "{last_weights}" --task test --name test_yolov7_100_to_1000

Namespace(weights=['D:\\OU Master AI\\Research methods\\Project\\YOLO_drone_detection\\results_100_augmented_to_1000\\weights\\last.pt'], data='D:\\OU Master AI\\Research methods\\Project\\YOLO_drone_detection\\custom.yaml', batch_size=6, img_size=640, conf_thres=0.001, iou_thres=0.65, task='test', device='0', single_cls=True, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project='runs/test', name='test_yolov7_100_to_1000', exist_ok=False, no_trace=False, v5_metric=False)
Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

                 all         430         465       0.879       0.751        0.82       0.442
Speed: 12.7/1.5/14.2 ms inference/NMS/total per 640x640 image at batch-size 6

Evaluating pycocotools mAP... saving runs\test\test_yolov7_100_to_1000\last_predictions.json...
pycocotoo

YOLOR  2023-5-7 torch 2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6143.5MB)

D:\OU Master AI\Research methods\Project\yolo-drones-env\Lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36481772 parameters, 6194944 gradients, 103.2 GFLOPS

Scanning images:   0%|          | 0/430 [00:00<?, ?it/s]
test: Scanning 'D:\OU Master AI\Research methods\Project\YOLO_drone_detection\test' images and labels... 158 found, 0 missing, 0 empty, 0 corrupted:  37%|###6      | 158/430 [00:00<00:00, 1567.39it/s]
test: Scanning 'D:\OU Master AI\Research methods\Project\YOLO_drone_detection\test' images and labels... 374 found, 0 missing, 0 empty, 0 corrupted:  87%|########6 | 374/430 [00:00<00:00, 1909.56it/s]
test: Scanning 'D:\OU Ma

## Create a collage image to show all data augmentations in a single image

In [56]:
def generate_collage(image_paths, output_path, size=(200, 200), margin=4):
    """
    Generate a collage of images.

    :param image_paths: A list of paths to the images.
    :param output_path: Path to save the collage.
    :param size: Size of each image (width, height).
    :param margin: Space between images.
    """

    assert len(image_paths) == 10, "There should be exactly 10 image paths."

    # Create a new image object, RGBA for supporting transparency
    collage = Image.new('RGBA', (5 * size[0] + 6 * margin, 2 * size[1] + 3 * margin), (105, 67, 65))

    # Iterate over the image paths and paste them into the collage
    for i, image_path in enumerate(image_paths):
        # Open the image file
        img = Image.open(image_path)
        # Resize it
        img = img.resize(size)
        # Calculate the position where to paste the image
        x = margin + (i % 5) * (size[0] + margin)
        y = margin + (i // 5) * (size[1] + margin)
        # Paste the image
        collage.paste(img, (x, y))

    # Save the collage
    collage.save(output_path)

In [57]:
image_folder = r'D:\OU Master AI\Research methods\Project\YOLO_drone_detection\augmented-dataset-validation-50-to-500'
images = ['pic_349.jpg', 'pic_349_jittered.jpg', 'pic_349_brightness_altered.jpg', 'pic_349_contrast_altered.jpg', 
          'pic_349_flipped_hori.jpg', 'pic_349_flipped_vert.jpg', 'pic_349_rotated_clockwise.jpg', 
          'pic_349_rotated_counterclockwise.jpg', 'pic_349_zoomed.jpg', 'pic_349_translated.jpg']
images = [os.path.join(image_folder, x) for x in images]
output_path = os.path.join(project_path, 'pic_349_collage.png')
generate_collage(images, output_path)